# Case Study: validation score isn't enough

In class, we mentioned that validation score often isn't an accurate measure of how well your model actually performs on new data. At first glance, you'd think that the validation set and test set are redundant -- if you do have a good validation score, shouldn't you also have a good test score? In this notebook, we'll explore an example where this is not the case. By doing so, we can begin to understand why the test set is so important.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## The data

Below is a function to generate data. It has 100 feature columns: x0,x1, x2, x3,...,x99, which are all random. There's also a goal column, y, which is `x0 + x1 + x2 + random noise)`. This means that y is related to only the first three columns, plus some random noise. The other 97 features features aren't related to y at all.

We create three variables: `x_train`, `x_valid`, `x_test` and the corresponding y-value variables.

Side note: In the function below, we're manually putting random noise into `y`. Real datasets will intrinsically contain random noise.

In [2]:
def gen_data(seed):
    np.random.seed(seed)
    data = []
    num_features = 100
    for i in range(500):
        row = [np.random.random() for i in range(num_features)]
        y = sum(row[:3]) + np.random.random()**2
        row.append(y)
        data.append(row)
    features = ['x'+str(i) for i in range(num_features)]
    all_columns = [i for i in features]
    all_columns.append('y')
    full_df = pd.DataFrame(data, columns=all_columns)
    features = full_df[features]
    goal = full_df['y']
    x_not_test,x_test,y_not_test,y_test = train_test_split(features, goal, test_size=0.5,random_state=seed)
    x_train,x_valid,y_train,y_valid = train_test_split(x_not_test, y_not_test, test_size=0.2,random_state=seed)
    return x_train,x_valid,x_test,y_train,y_valid,y_test 

In [3]:
# The 42 makes it so we get the same dataset each time.
# If you want to play around, you can change 42 to another number to get a different random dataset.
x_train,x_valid,x_test,y_train,y_valid,y_test = gen_data(42)

In [4]:
print("train set size:",len(y_train))
print("validation set size:",len(y_valid))
print("test set size:",len(y_test))

train set size: 200
validation set size: 50
test set size: 250


## The models
We're going to use decision tree regressors to predict `y` using `x1`,`x2`,`x3`, and `x4`. Don't worry if you don't know what a decision tree is. All we need to know is that decision trees are models that have a hyperparameter called `max_depth`, which will control how complex the model is and how much it uses each feature.

We're going to use a fully automated this process. We'll build a bunch of decision trees with different `max_depth`s. Then, we'll choose the best of these.

What does "best" mean? We'll try two definitions of "best": <br>
(1) highest validation score<br>
(2) lowest (bias^2 + variance)

### Algorithm 1: maximize validation scores
Below is some code to go through a bunch of `max_depth`s, and choose the one that gives the model with the highest validation score. You don't have to worry about the details of the code if you don't want to -- that's not the focus here.

There's one thing you should take note of though: we don't use `x_test` or `y_test`. Those are only to be used at the very end. In the following block of code, we simply use the train and validation sets over and over again.

In [5]:
depths, valid_scores = [],[]
for max_depth in range(1,1000):
    tree = DecisionTreeRegressor(max_depth=max_depth)
    tree.fit(x_train,y_train)
    depths.append(max_depth)
    valid_scores.append(tree.score(x_valid,y_valid))
best_index = np.argmax(valid_scores)
best_depth,best_score = (depths[best_index], valid_scores[best_index])

print("highest validation score of",best_score,"at a max_depth of",best_depth)
print("---------------------")
tree_using_valid_score = DecisionTreeRegressor(max_depth = best_depth)
tree_using_valid_score.fit(x_train,y_train)
print("test score:",tree_using_valid_score.score(x_test,y_test))

highest validation score of 0.32045044860584704 at a max_depth of 633
---------------------
test score: 0.2047028476997227


The validation and test scores are quite different! Here, we "trust" test score over validation score. The only reason our validation score is higher is because we looked at a bunch of models and picked out the one with the highest validation score.

We'll come back to why our model has high validation score but low test score. Let's first move onto our second algorithm.

### Algorithm 2: minimize (bias^2 + variance)
Here, we try a bunch of `max_depth`s and choose the best one we can find, where "best" means "lowest bias^2+variance". In other words, "lowest total error".<br>
Again, you don't have to worry about the details of the code below if you don't want to -- the complexity of code in the next two cells is out of the scope of this class. And again, we don't use `x_test` and `y_test`.

In [6]:
def estimate_variance(model,x_train,y_train,x_valid):
    assert len(x_train) == len(y_train)
    iters = 100
    preds = []
    all_indices = list(range(len(x_train)))
    x_train = np.array(x_train)
    y_train = np.array(x_train)
    for _ in range(iters):
        subset_of_indices = np.random.choice(all_indices,len(all_indices)//2)
        subset_x_train, subset_y_train = x_train[subset_of_indices],y_train[subset_of_indices]
        model.fit(subset_x_train, subset_y_train)
        preds.append(model.predict(x_valid))
    variances = [np.var([pred[i] for pred in preds]) for i in range(len(x_valid))]
    return np.mean(variances)

In [7]:
best_depth,lowest_error = -1,1e20
print("Running Algorithm 2. This might take a minute or two...")
lower = 1
upper = 1000
while True:
    depths, errors = [],[]
    for max_depth in range(lower,upper+1,max((upper - lower) // 10,1)):
        tree = DecisionTreeRegressor(max_depth=max_depth,random_state=2)
        tree.fit(x_train,y_train)
        valid_pred = tree.predict(x_valid)
        bias_squared = mean_squared_error(valid_pred,y_valid)
        variance = estimate_variance(tree,x_train,y_train,x_valid)
        depths.append(max_depth)
        errors.append(bias_squared + variance)
    best_index = np.argmin(errors)
    best_depth,lowest_error = depths[best_index], errors[best_index]
    new_lower = depths[max(0, best_index - 1)]
    new_upper = depths[min(len(depths)-1,best_index + 1)]
    if new_lower == lower and new_upper == upper:
        break
    lower = new_lower
    upper = new_upper
print("lowest error of",lowest_error,"at a max_depth of",best_depth)
print("---------------------")
tree_using_bias_variance = DecisionTreeRegressor(max_depth = best_depth)
tree_using_bias_variance.fit(x_train,y_train)
print("test score:",tree_using_bias_variance.score(x_test,y_test))

Running Algorithm 2. This might take a minute or two...
lowest error of 0.2215128341597164 at a max_depth of 3
---------------------
test score: 0.43215759658266695


Our test score is a lot higher for Algorithm 2 than for Algorithm 1!

(Also, validation and test scores are again quite far apart.)

## Why was using validation score less effective than using bias&variance?

To begin to answer this question, let's look at how much each model weighted each feature. In other words, when each model was making a prediction, how important was x1 in deciding the prediction? x2? x3? What about x4 through x99?

Take a look at the feature_importances_ attribute of DecisionTreeRegressor below.

In [8]:
DecisionTreeRegressor.feature_importances_?

In [9]:
print("Feature importances for tree using validation score:")
print("x0",tree_using_valid_score.feature_importances_[0])
print("x1",tree_using_valid_score.feature_importances_[1])
print("x2",tree_using_valid_score.feature_importances_[2])
print("sum of all other features importances:", sum(tree_using_valid_score.feature_importances_[3:]))

Feature importances for tree using validation score:
x0 0.2436866479598124
x1 0.16681696655265588
x2 0.2533512596117346
sum of all other features importances: 0.336145125875797


In [10]:
print("Feature importances for tree using bias&variance:")
print("x0",tree_using_bias_variance.feature_importances_[0])
print("x1",tree_using_bias_variance.feature_importances_[1])
print("x2",tree_using_bias_variance.feature_importances_[2])
print("sum of all other features importances:", sum(tree_using_bias_variance.feature_importances_[3:]))

Feature importances for tree using bias&variance:
x0 0.3266773504966206
x1 0.2623437138331212
x2 0.36422046415681575
sum of all other features importances: 0.0467584715134424


Recall that we defined our function to be `y = x0 + x1 + x2 +(random noise)`. So, <b>y only depended on the first three features</b> (plus some random noise).

When we used bias and variance, we got what we'd expect: x1, x2, and x3 are treated around equally important, and the other 97 features combined only contribute t oaround 5% of the prediction.

When we used validation score, the 97 unrelated features contribute to around a third of the prediction! That's what causes the high-validation model to perform badly on the test set.

Now, the question is: <b>why did minimizing validation score result in giving unrelated features such high weights?</b> Keep in mind, we really did minimize validation score; it's just that minimizing validation score did not minimize test score.<br>
<b>The answer</b>: Because we found the *absolute* minimum of the validation score, we captured random relationships between x3 through x99 and and y that just happened to exist between the train and validation sets. In other words, we captured relationships between the *random noise* of x3 through x99 and y -- not actual relationships between the variables themselves.<br>
To understand how this happened, imagine for a moment that you're a computer. You're trying to maximize validation score, and you've reached a point where you only use x0, x1, and x2 to predict y. Now, you wonder, "Can I do better?" You try some more things, and realize that if you use a few more features, your score increases by 0.00001. At that point, a data scientist would realize, "That would be using more features in our model, which increases model complexity! <b>That adds a lot of variance to the model for such a measly increase in score</b>". But the computer doesn't know this. The computer happily chooses the model that uses lots of features for that 0.00001 increase in score.<br>
As a result, the computer spits out a model that captures these fake relationships between x3 through x99 and y. These "relationships" aren't actual relationships. It's just a coincidence -- in the train and validation sets, the random noise in x3 happened to look similar to the random noise in y, and same with x4 through x99. Because this coincidence is specific to the noise in the train and validation sets, the model starts performing badly on the test set.

## So...when I'm evaluating models, should I use bias and variance instead of validation score?
Calculating/estimating variance is kind of a pain, and takes a lot of computing time. Luckily, you usually don't have to find the variance. There is a set of techniques called *cross-validation* (lectures 7 and 8) that lets you avoid manually calculating variance.

## Summary
Validation score wasn't a clear indicator of test score -- maximizing validation score clearly did not maximize test score.

This tells us that validation score alone isn't enough. We could have a really great validation score, but still do badly on new data. This is often the case when we use the validation set as part of our model-selection algorithm. For example, in Algorithm 1, we used validation score to choose between several models.<br>
<b>In order to accurately estimate how well our model performs on new data, we need the test set.</b><br>
We can trust the test set because we *never* use it in our model-selection algorithm; we only use it for evaluation.